# Example of correct bulk upload using python elasticsearch client

[Reference blog](http://unroutable.blogspot.com/2015/03/quick-example-elasticsearch-bulk-index.html).

[elasticsearch-py](http://elasticsearch-py.readthedocs.org/en/latest/helpers.html) documentation



In [16]:
import itertools
import string
from elasticsearch import Elasticsearch,helpers
es = Elasticsearch()

In [17]:
k = ({'_type':'foo', '_index':'test','letters':''.join(letters)}
     for letters in itertools.permutations(string.letters,2))


In [18]:
k.next()

{'_index': 'test', '_type': 'foo', 'letters': 'AB'}

In [19]:
k.next()

{'_index': 'test', '_type': 'foo', 'letters': 'AC'}

In [20]:
#es.indices.create('test')

In [21]:
helpers.bulk(es,k)

(2650, [])

In [22]:
es.count(index='test')

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5}, u'count': 25129}

# Modifying the example to fit our use case

We are going to start by reading the json file returned from our search api

In [23]:
import json
from elasticsearch import Elasticsearch
from greencall.utils.loadelastic import (read_json, load_elastic,
                                         prepare_all_documents,
                                        map_documents)
from greencall.csvclean.inputCsv import read_csv

In [24]:
resultspath = 'examples/finance_demo2.json'
esformat = {
            "_index": "customsearch",
            "_type": "website",
            "_id": None,
            "_source": ""
        }

accountdict = 'examples/finance_demo.csv'

In [25]:
es = Elasticsearch()

In [26]:
results = read_json(resultspath)

In [27]:
len(results)

40

In [28]:
#actions = prepare_all_documents(results, esformat, accountdict)
for key in results.keys():
    key = int(key)
    print("key: {}, type: {}".format(key, type(key)))

key: 11112288, type: <type 'int'>
key: 11112382, type: <type 'int'>
key: 11111346, type: <type 'int'>
key: 11112662, type: <type 'int'>
key: 11112282, type: <type 'int'>
key: 11111135, type: <type 'int'>
key: 11111709, type: <type 'int'>
key: 11111836, type: <type 'int'>
key: 11111815, type: <type 'int'>
key: 11111973, type: <type 'int'>
key: 11111626, type: <type 'int'>
key: 11112010, type: <type 'int'>
key: 11111972, type: <type 'int'>
key: 11111337, type: <type 'int'>
key: 11112210, type: <type 'int'>
key: 11111143, type: <type 'int'>
key: 11111146, type: <type 'int'>
key: 11111437, type: <type 'int'>
key: 11111253, type: <type 'int'>
key: 11111417, type: <type 'int'>
key: 11111774, type: <type 'int'>
key: 11111169, type: <type 'int'>
key: 11111294, type: <type 'int'>
key: 11111757, type: <type 'int'>
key: 11112584, type: <type 'int'>
key: 11111821, type: <type 'int'>
key: 11111336, type: <type 'int'>
key: 11111804, type: <type 'int'>
key: 11111495, type: <type 'int'>
key: 11111157,

In [29]:
actions = prepare_all_documents(results, esformat, read_csv(accountdict))

key not found: 11112288
0
key not found: 11112382
0
0
key not found: 11112662
0
key not found: 11112282
0
0
0
key not found: 11111836
0
key not found: 11111815
0
key not found: 11111973
0
15
key not found: 11112010
15
key not found: 11111972
15
30
key not found: 11112210
30
44
59
70
85
100
key not found: 11111774
100
114
128
139
key not found: 11112584
139
key not found: 11111821
139
150
key not found: 11111804
150
164
179
key not found: 11112260
179
key not found: 11112028
179
key not found: 11112556
179
194
key not found: 11112269
194
key not found: 11112349
194
key not found: 11112630
194
205
220
234


In [30]:
actions = iter(actions)

In [31]:
actions.next()

{'_id': 11,
 '_index': 'customsearch',
 '_source': {'account_holder': 'Billy Masters',
  'account_number': u'11111626',
  u'url': [u'type', u'template']},
 '_type': 'website'}

In [32]:
actions.next()

{'_id': 12,
 '_index': 'customsearch',
 '_source': {'account_holder': 'Billy Masters',
  'account_number': u'11111626',
  u'type': u'application/json'},
 '_type': 'website'}

In [33]:
actions.next()

{'_id': 13,
 '_index': 'customsearch',
 '_source': {'account_holder': 'Billy Masters',
  'account_number': u'11111626',
  u'totalResults': u'4640'},
 '_type': 'website'}

In [34]:
actions.next()

{'_id': 14,
 '_index': 'customsearch',
 '_source': {'account_holder': 'Billy Masters',
  'account_number': u'11111626',
  u'title': u'greencall'},
 '_type': 'website'}

In [36]:
helpers.bulk(es,actions)

BulkIndexError: ('2 document(s) failed to index.', [{u'index': {u'status': 400, u'_type': u'website', u'_id': u'22', u'error': u'MapperParsingException[failed to parse [items.pagemap.review.datepublished]]; nested: MapperParsingException[failed to parse date field [6/7/15], tried both date format [dateOptionalTime], and timestamp number with locale []]; nested: IllegalArgumentException[Invalid format: "6/7/15" is malformed at "/7/15"]; ', u'_index': u'customsearch'}}, {u'index': {u'status': 400, u'_type': u'website', u'_id': u'26', u'error': u'MapperParsingException[failed to parse [items.pagemap.review.datepublished]]; nested: MapperParsingException[failed to parse date field [08/08/2013], tried both date format [dateOptionalTime], and timestamp number with locale []]; nested: IllegalArgumentException[Invalid format: "08/08/2013" is malformed at "/08/2013"]; ', u'_index': u'customsearch'}}])

In [37]:
es.count(index='customsearch')

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5}, u'count': 40}